In [ ]:
!pip install googledrivedownloader

In [ ]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.eager.python import tfe
from PIL import Image

tf.disable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [ ]:
gdd.download_file_from_google_drive(file_id='1ycR7Aexe8xbZ8oEDsQwGc9SIiFklRpfu', dest_path='./data.zip', unzip=True)
data_dir = 'data'

train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [ ]:
# Data Generator dùng cho training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Data Generator dùng cho validation và testing
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 24

# ImageDataGenerator chỉ chấp nhận kiểu label là string, 
# ta chuyển cột label sang string
train_df["label"] = train_df["label"].map(lambda x: str(x))
# Chia dữ liệu thành train và validation
train_data, val_data = train_test_split(train_df, test_size=0.25)

# Tạo luồng dữ liệu cho quá trình train, các bạn có thể tìm hiểu các tham số ở Keras document
train_gen = train_datagen.flow_from_dataframe(dataframe=train_data, 
                                        directory="data/images", 
                                        x_col="image", 
                                        y_col="label",
                                        class_mode="categorical",
                                        target_size=(224,224), 
                                        batch_size=batch_size)

# Tạo luồng dữ liệu cho quá trình test
val_gen = test_datagen.flow_from_dataframe(dataframe=val_data, 
                                        directory="data/images", 
                                        x_col="image", 
                                        y_col="label",
                                        class_mode="categorical",
                                        shuffle=False,
                                        target_size=(224,224), 
                                        batch_size=batch_size)

test_gen = test_datagen.flow_from_dataframe(dataframe=test_df, 
                                        directory="data/images", 
                                        x_col="image",
                                        class_mode=None,
                                        shuffle=False,
                                        batch_size=32,
                                        target_size=(224,224))

In [ ]:
!pip install efficientnet
from efficientnet.keras import EfficientNetB3

In [ ]:
import keras.backend as K
import keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# Số lượng training step mỗi epoch
steps_per_epoch = train_gen.n // batch_size
# Số lượng validation step
validation_steps = val_gen.n // batch_size

# Khởi tạo base model Resnet với pretrained weights từ ImageNet
#tf.keras.applications.mobilenet.MobileNet
#base_model = tf.keras.applications.mobilenet.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Tạo model hoàn chỉnh bằng cách thêm lớp classifier
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
predictions = keras.layers.Dense(11, activation='softmax')(x)
model = keras.models.Model(input = base_model.input, output = predictions)

# Freeze các lớp CNN ban đầu
base_model.trainable = False

# Tạo callback để lưu model có accuracy trên tập validation tốt nhất
mcp = keras.callbacks.ModelCheckpoint("efficientnetb3.h5", monitor="val_f1",
                  save_best_only=True, mode='max', save_weights_only=True, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_f1', factor=0.1, mode='max', patience=5, min_lr=1e-8, verbose=1)

# Compile model
model.compile(optimizer=keras.optimizers.Adam(0.01), loss='categorical_crossentropy',
              metrics=[f1])

#model.load_weights("efficientnetb3.h5")

# Huấn luyện 10 epochs với learning rate lớn
model.fit_generator(train_gen, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=10,
                    verbose=1, 
                    validation_data=val_gen,
                    validation_steps=validation_steps, 
                    callbacks=[mcp, rlr])

# Unfreeze toàn bộ mạng
base_model.trainable = True

# Compile model
model.compile(optimizer=keras.optimizers.Adam(0.001), loss='categorical_crossentropy',
              metrics=[f1])



# Huấn luyện 100 epochs với learning rate nhỏ
model.fit_generator(train_gen, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=100,
                    verbose=1, 
                    validation_data=val_gen,
                    validation_steps=validation_steps, 
                    callbacks=[mcp])

In [ ]:
model.load_weights("efficientnetb3.h5")
pred = model.predict_generator(test_gen)

# pred là một ma trận xác suất của ảnh trên các lớp.
# Ta lấy lớp có xác suất cao nhất trên từng ảnh bằng hàm argmax
predicted_class_indices = np.argmax(pred, axis=1)

# Dictionary ánh xạ nhãn và index
labels = train_gen.class_indices

# Ánh xạ indices về nhãn đúng
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
test_df['label'] = predictions
test_df.to_csv("submission.csv", index=False)

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')